This script takes in data saved in the local postgres database used for this project and saves it into one large HDF5 file.


In [41]:
# imports 
import sys
import os
import numpy as np
import psycopg2
import pickle
import logging
import dask.dataframe as dd
from sqlalchemy import create_engine
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
from dotenv import dotenv_values
import hdf5storage
import h5py
from sys import getsizeof
config = dotenv_values("../.env")

In [43]:
def get_db_connection():
    conn = psycopg2.connect(
        dbname='simDB',
        user='user',
        password='password',
        host='localhost',
        port='5432'
    )
    return conn

def fetch_trial_data(conn, trial_id,n=1,limit=500):
    try:
        cur = conn.cursor()
        
        # Fetch data for the given trial_id
        query = '''
        SELECT timestep, simulation_time, input_data, output_data, state_data
        FROM simulation_data
        WHERE trial_id = %s AND MOD(timestep, %s) = 0
        ORDER BY timestep ASC LIMIT %s;
        '''
        cur.execute(query, (trial_id, n, limit))
        rows = cur.fetchall()
        
        # Deserialize data
        data = []
        for row in rows:
            timestep, simulation_time, input_data_bin, output_data_bin, state_data_bin = row
            input_data = np.array(pickle.loads(input_data_bin)).flatten()
            output_data = np.array(pickle.loads(output_data_bin)).flatten()
            state_data = np.array(pickle.loads(state_data_bin)).flatten()
            data.append((timestep, simulation_time, input_data, output_data, state_data))
        
        cur.close()
        return data
    except Exception as e:
        print(f"Error fetching data for trial_id {trial_id}: {e}")
        return None

In [44]:
# Trial Parameters  
timesteps = 1000
trials = 40
# Make list of trial ids from 67-108 skipping 79 and 87
trial_ids = list(range(67,79)) + list(range(80,87)) + list(range(88,109)) #[67]
n = 1 # Downsampling factor

In [45]:
print(trial_ids)
print(len(trial_ids))

[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]
40


In [46]:
# Read a single timestep of data from the database to get dimensions of stuff correct
conn = get_db_connection()
data = fetch_trial_data(conn, trial_ids[0], n, 1)
conn.close()
timestep, simulation_time, input_data, output_data, state_data = data[0]
outputDim = np.size(output_data)
inputDim = np.size(input_data)
stateDim = np.size(state_data)
print(f"OutputDim: {outputDim}, InputDim: {inputDim}, StateDim: {stateDim}")



OutputDim: 40, InputDim: 6, StateDim: 243789


In [47]:
states = np.zeros((stateDim, timesteps, trials))
inputs = np.zeros((inputDim, timesteps, trials))
outputs = np.zeros((outputDim, timesteps, trials))
simulation_times = np.zeros((timesteps, trials))
simulation_timesteps = np.zeros((timesteps, trials))
print(getsizeof(states)//(1024*1024*1024), "GB")

72 GB


In [48]:
# Iterate over trials and fetch data
for i, trial_id in enumerate(trial_ids):

    print(f"Fetching data for trial_id {trial_id}")
    conn = get_db_connection()
    trial_data = fetch_trial_data(conn, trial_id,n,limit=timesteps)
    conn.close()
    
    # Extract output data
    for timestep, simulation_time, input_data, output_data, state_data in trial_data[:timesteps]:
        outputs[:,timestep,i] = output_data
        inputs[:,timestep,i] = input_data
        states[:,timestep,i] = state_data
        simulation_times[timestep,i] = simulation_time
        simulation_timesteps[timestep,i] = timestep

Fetching data for trial_id 67
Fetching data for trial_id 68
Fetching data for trial_id 69
Fetching data for trial_id 70
Fetching data for trial_id 71
Fetching data for trial_id 72
Fetching data for trial_id 73
Fetching data for trial_id 74
Fetching data for trial_id 75
Fetching data for trial_id 76
Fetching data for trial_id 77
Fetching data for trial_id 78
Fetching data for trial_id 80
Fetching data for trial_id 81
Fetching data for trial_id 82
Fetching data for trial_id 83
Fetching data for trial_id 84
Fetching data for trial_id 85
Fetching data for trial_id 86
Fetching data for trial_id 88
Fetching data for trial_id 89
Fetching data for trial_id 90
Fetching data for trial_id 91
Fetching data for trial_id 92
Fetching data for trial_id 93
Fetching data for trial_id 94
Fetching data for trial_id 95
Fetching data for trial_id 96
Fetching data for trial_id 97
Fetching data for trial_id 98
Fetching data for trial_id 99
Fetching data for trial_id 100
Fetching data for trial_id 101
Fetching

In [49]:
# Check whats in the state matrix
print(states[:,1,0])

[-1.48000000e+02 -5.21609373e-05  4.99955557e+01 ... -4.09833327e+02
 -1.70334065e+01 -1.78415533e+01]


In [50]:
outfileNamehdf5 = config["currentDirectory"] +"data/archivedDataSets/ContiguousAssembly/FreqSweepDataset.hdf5"
with h5py.File(outfileNamehdf5, 'w') as f:
    f.create_dataset('stateData', data=states,maxshape=(None,None,None),chunks=(4096,4096,1))
    f.create_dataset('inputData', data=inputs,maxshape=(None,None,None),chunks=(4096,4096,1))
    f.create_dataset('outputData', data=outputs,maxshape=(None,None,None),chunks=(4096,4096,1))
    f.create_dataset('simulationTimes', data=simulation_times,maxshape=(None,None),chunks=(4096,1))
    f.create_dataset('simulationTimesteps', data=simulation_timesteps,maxshape=(None,None),chunks=(4096,1))

In [61]:
# Check on contents of written file in trial 12
outfileNamehdf5 = config["currentDirectory"] +"data/archivedDataSets/ContiguousAssembly/FreqSweepDataset.hdf5"
with h5py.File(outfileNamehdf5, 'r') as f:
    print(f.keys())
    # print(f['stateData'][:,:,14])
    print(f['inputData'][:,:,12])
    # print(f['outputData'][:,:,12])
    # print(f['simulationTimes'][:,12])
    # print(f['simulationTimesteps'][:,12])

<KeysViewHDF5 ['inputData', 'outputData', 'simulationTimes', 'simulationTimesteps', 'stateData']>
[ 0.01732051  0.01699785  0.01665842  0.01630256  0.0159306   0.01554292
  0.0151399   0.01472194  0.01428945  0.01384286  0.01338261  0.01290915
  0.01242296  0.0119245   0.01141427  0.01089278  0.01036054  0.00981808
  0.00926592  0.00870462  0.00813473  0.00755682  0.00697144  0.00637919
  0.00578064  0.00517638  0.00456702  0.00395315  0.00333537  0.00271431
  0.00209057  0.00146476  0.00083751  0.00020944 -0.00041885 -0.00104672
 -0.00167356 -0.00229874 -0.00292166 -0.00354169 -0.00415823 -0.00477067
 -0.0053784  -0.00598082 -0.00657733 -0.00716736 -0.00775031 -0.00832562
 -0.0088927  -0.00945102 -0.01       -0.01053912 -0.01106783 -0.01158562
 -0.01209198 -0.01258641 -0.01306841 -0.01353752 -0.01399327 -0.0144352
 -0.0148629  -0.01527592 -0.01567387 -0.01605635 -0.01642298 -0.01677341
 -0.01710729 -0.01742428 -0.01772407 -0.01800638 -0.01827091 -0.01851741
 -0.01874564 -0.01895537 -0